<a href="https://colab.research.google.com/github/anuragbisht12/NLP/blob/master/NER_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
from collections import defaultdict
from spacy import displacy
import pandas as pd

texts = ["When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."
         ,"Twitter will allow employees to work remotely for the rest of this year."
        ,'It is really hot in Delhi these days.']
        # ,'Elon Musk the founder and CEO of SpaceX']

SCORE=0.1
# Number of alternate analyses to consider. More is slower, and not necessarily better -- you need to experiment on your problem.
beam_width = 16
# This clips solutions at each step. We multiply the score of the top-ranked action by this value, and use the result as a threshold. This prevents the parser from exploring options that look very unlikely, saving a bit of efficiency. Accuracy may also improve, because we've trained on greedy objective.
beam_density = 0.0001 
nlp = spacy.load('en_core_web_sm')
render_sentences = list(nlp.pipe(texts))
print("\n"+"*"*180+"\n")
for sent in render_sentences: 
    displacy.render(sent, style='ent', jupyter=True)
    print("\n"+"*"*180+"\n")

with nlp.disable_pipes('ner'):
    docs = list(nlp.pipe(texts))
beams = nlp.entity.beam_parse(docs, beam_width=beam_width, beam_density=beam_density)
df=pd.DataFrame(columns=['Entity', 'Name', 'Score'])
i=0
for doc, beam in zip(docs, beams):
    entity_scores = defaultdict(float)
    for score, ents in nlp.entity.moves.get_beam_parses(beam):
        if score>SCORE:
            for start, end, label in ents:       
                entity_scores[(start, end, label,doc[start:end])] += score
    for key,val in entity_scores.items():
        df.loc[i] = [key[2],' '.join([str(i) for i in key[3]]),round(val*100,2)]
        i+=1

print(df)


************************************************************************************************************************************************************************************




************************************************************************************************************************************************************************************




************************************************************************************************************************************************************************************




************************************************************************************************************************************************************************************

  Entity             Name  Score
0    ORG  Sebastian Thrun  84.68
1    ORG           Google  97.60
2   DATE             2007  97.60
3   NORP        Sebastian  12.92
4   DATE        this year  99.70
5    GPE            Delhi  98.96
6   DATE       these days  98.96
